In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# For static images:
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5)

names = ["p", "s", "r"]
file_list = []
result = []

for i in names:
    for j in range(1,31):
        file_list.append(i + str(j) + ".png")
        
print(len(file_list))

for idx, file in enumerate(file_list):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.multi_hand_landmarks:
        continue


    image_hight, image_width, _ = image.shape
    annotated_image = image.copy()
    
    output = []

    for hand_landmarks in results.multi_hand_landmarks:
        for item in hand_landmarks.landmark:
            output.append(item.x)
            output.append(item.y)
            output.append(item.z)
            
    result.append(output)

hands.close()


answer = []

for i in range(1,4):
    for j in range(30):
        answer.append(i)
        
print(len(result))

In [ ]:
X_train = np.transpose(result)
y_train = answer

inverse = np.linalg.inv(np.matmul(X_train,np.transpose(X_train)))
rest = np.matmul(y_train,np.transpose(X_train))
W = np.matmul(rest,inverse)

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# For static images:
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5)



test = "s6.png"
image = cv2.flip(cv2.imread(test), 1)
# Convert the BGR image to RGB before processing.
results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


image_hight, image_width, _ = image.shape
annotated_image = image.copy()

output2 = []

for hand_landmarks in results.multi_hand_landmarks:
    for item in hand_landmarks.landmark:
        output2.append(item.x)
        output2.append(item.y)
        output2.append(item.z)

print(output2)

In [ ]:
output_test = np.transpose(output2)
res1 = np.round(np.matmul(np.transpose(W),X_train))

print(res1)

In [ ]:
# For webcam input:
hands = mp_hands.Hands(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture(0)
while cap.isOpened():
  success, image = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    # If loading a video, use 'break' instead of 'continue'.
    continue

  # Flip the image horizontally for a later selfie-view display, and convert
  # the BGR image to RGB.
  image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
  image.flags.writeable = False
  results = hands.process(image)

  result3 = []
  # Draw the hand annotations on the image.
  image.flags.writeable = True
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        output3 = []
        mp_drawing.draw_landmarks(
        image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        for item in hand_landmarks.landmark:
            output3.append(item.x)
            output3.append(item.y)
            output3.append(item.z)
        
        output_test2 = np.transpose(output3)
        res2 = np.round(np.matmul(np.transpose(W),output_test2))

        print(res2)
    
  cv2.imshow('MediaPipe Hands', image)
  if cv2.waitKey(5) & 0xFF == 27:
    break
hands.close()
cap.release()